In [15]:
import os 
import random

import numpy as np
import pandas as pd 
import ujson

In [26]:
run_path = "/home/jupyter/jointly_rec_and_search/datasets/rec_search/search/runs/bm25.train.run" 
qrels_path = "/home/jupyter/jointly_rec_and_search/datasets/rec_search/search/qrels.train.tsv" 
out_path = "/home/jupyter/jointly_rec_and_search/datasets/rec_search/search/1pos_1neg.train.tsv"

#neg_num = 1
#topk = 1000
qid_to_relpid = {}
rel_df = pd.read_csv(qrels_path, sep="\t", names=["qid", "q0", "pid", "score"])
for qid, group in rel_df.groupby("qid"):
    relpid = group.sample(1).iloc[0]["pid"]
    qid_to_relpid[qid] = relpid

train_examples = []
df = pd.read_csv(run_path, sep=" ", names=["qid", "q0", "pid", "rank", "score", "model_name"])
for qid, group in df.groupby("qid"):
    if len(group) <= 10:
        continue 
    relpid = qid_to_relpid[qid]
    negpid = group.sample(1).iloc[0]["pid"]
    train_examples.append((qid, relpid, negpid))

print("number of train_examples = {}".format(len(train_examples)))

with open(out_path, "w") as fout:
    for exp in train_examples:
        qid, relpid, negpid = exp 
        fout.write(f"{qid}\t{relpid}\t{negpid}\n")
    

number of train_examples = 183380


In [27]:
! wc -l $qrels_path
! head -n 5 $qrels_path
print(50*"=")
! wc -l $out_path
! head -n 5 $out_path

762284 /home/jupyter/jointly_rec_and_search/datasets/rec_search/search/qrels.train.tsv
0	Q0	51832	1
1	Q0	2126637	1
2	Q0	1691797	1
3	Q0	1768047	1
4	Q0	26111	1
183380 /home/jupyter/jointly_rec_and_search/datasets/rec_search/search/1pos_1neg.train.tsv
2	1691797	2156531
3	1768047	2155041
4	26111	1121933
5	220932	879053
6	85698	656731
